In [ ]:
from semantic_kernel import Kernel

kernel = Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106"),
)

In [ ]:
# note: using plugins from the samples folder
plugins_directory = "./plugins"

funFunctions = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="FunPlugin")

jokeFunction = funFunctions["Joke"]

In [ ]:
# result = await kernel.invoke(jokeFunction, input="travel to dinosaur age", style="silly")
# print(result)

In [ ]:
from semantic_kernel.contents.chat_history import ChatHistory  # noqa: F401
from semantic_kernel.functions.kernel_arguments import KernelArguments  # noqa: F401
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.core_plugins.text_plugin import TextPlugin
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai

shakespeare_func = KernelFunctionFromPrompt(
    function_name="Shakespeare",
    plugin_name="WriterPlugin",
    prompt="""
{{$input}}

Rewrite the above in the style of Shakespeare.
""",
    prompt_execution_settings=sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.8,
    ),
    description="Rewrite the input in the style of Shakespeare.",
)
yoda_func = KernelFunctionFromPrompt(
    function_name="Yoda",
    plugin_name="WriterPlugin",
    prompt="""
{{$input}}

Rewrite the above in the style of Yoda.
""",
    prompt_execution_settings=sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.8,
    ),
    description="Rewrite the input in the style of Yoda.",
)
kernel.add_function(plugin_name="WriterPlugin", function=shakespeare_func)
kernel.add_function(plugin_name="WriterPlugin", function=yoda_func)

for plugin_name, plugin in kernel.plugins.items():
    for function_name, function in plugin.functions.items():
        print(f"Plugin: {plugin_name}, Function: {function_name}")

In [ ]:
from semantic_kernel.planners import SequentialPlanner

ask = """
Tell me a joke in Shakespeare style"""

planner = SequentialPlanner(kernel, service_id)
sequential_plan = await planner.create_plan(goal=ask)
print("The plan's steps are:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

result = await sequential_plan.invoke(kernel)
print(result)

In [ ]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin

class OrderInfoPlugin:
    """
    Description: OrderInfoPlugin provides a set of functions to view order information.
    """

    @kernel_function(name="GetOrderList", description="Given a user_id return a list of orders")
    def send_email(
        self,
        user_id: Annotated[str, "the user id"],
    ) -> Annotated[list[dict], "the list of orders"]:
        """Get list of order that belong to the user"""
        orders = [
            {"order_id": "NN-001", "status": "payment_pending", "price": 100},
            {"order_id": "NN-002", "status": "in_transit", "price": 50},
        ]
        return orders

    @kernel_function(name="GetOrderDetail", description="Given a user_id and an order_number return the order details")
    def get_email_address(
        self,
        user_id: Annotated[str, "the user id"],
        order_number: Annotated[str, "the order number"],
    ) -> Annotated[dict, "the order details"]:
        """Get order details"""
        order = {"order_id": order_number, 
                 "status": "in_transit", 
                 "order_placed_date": "2022-01-01",
                 "sale_orders": [
                     {"sku_number": "1234", "quantity": 2, "price": 50, "status": "in_transit", "seller_name": "seller_1"},
                     {"sku_number": "1235", "quantity": 2, "price": 50, "status": "in_transit", "seller_name": "seller_2"},
                 ],
                 "price": 100}
        return order
    
kernel.add_plugin(plugin_name="OrderInfoPlugin", plugin=OrderInfoPlugin())
kernel.add_plugin(plugin_name="MathPlugin", plugin=MathPlugin())
kernel.add_plugin(plugin_name="TimePlugin", plugin=TimePlugin())

In [ ]:
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)

question = "Get the order list for user_id: 1234"

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000,
)

planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)

result = await planner.invoke(kernel, question)
print(f"Chat history: {result.chat_history}\n")
print(f"Q: {question}\nA: {result.final_answer}\n")


In [ ]:
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.prompt_template import PromptTemplateConfig

prompt = """
{{$history}}
User: {{$user_input}}
ChatBot: """

execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are E-Commerce chatbot who can help customer about their order.")

async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [ ]:
await chat("My user id is 1234, get my order list?")